# 🧪 Poldantic Demo: Convert Pydantic Models to Polars Schemas

In [1]:
from pydantic import BaseModel
from typing import Optional, List
import polars as pl
import poldantic as pd

## 🧩 Define nested Pydantic models

In [2]:
class Address(BaseModel):
    street: str
    city: str
    zip_code: Optional[int]

class User(BaseModel):
    id: int
    name: str
    email: Optional[str]
    is_active: bool
    address: Address
    tags: List[str]

## 📜 Polars Schema Outputs

In [3]:
from pprint import pprint

schema_dict = pd.get_polars_schema_dict(User)
print("Polars schema dict:")
pprint(schema_dict)

schema_obj = pd.get_polars_schema(User)
print("\nPolars schema (pl.Schema):")
pprint(schema_obj)

struct_dtype = pd.get_polars_struct(User)
print("\nPolars Struct dtype:")
pprint(struct_dtype)

fieldinfo = pd.get_polars_fieldinfo_dict(User)
pprint("\nFieldInfo dict:")
for name, info in fieldinfo.items():
    print(f"{name}: {info}")

Polars schema dict:
{'address': Struct({'street': String, 'city': String, 'zip_code': Int64}),
 'email': String,
 'id': Int64,
 'is_active': Boolean,
 'name': String,
 'tags': List(String)}

Polars schema (pl.Schema):
Schema([('id', Int64),
        ('name', String),
        ('email', String),
        ('is_active', Boolean),
        ('address',
         Struct({'street': String, 'city': String, 'zip_code': Int64})),
        ('tags', List(String))])

Polars Struct dtype:
Struct({'id': Int64, 'name': String, 'email': String, 'is_active': Boolean, 'address': Struct({'street': String, 'city': String, 'zip_code': Int64}), 'tags': List(String)})
'\nFieldInfo dict:'
id: FieldInfo(dtype=Int64, nullable=False)
name: FieldInfo(dtype=String, nullable=False)
email: FieldInfo(dtype=String, nullable=True)
is_active: FieldInfo(dtype=Boolean, nullable=False)
address: FieldInfo(dtype=Struct({'street': String, 'city': String, 'zip_code': Int64}), nullable=False)
tags: FieldInfo(dtype=List(String), nullab

## 📊 Create a Polars DataFrame and show inferred schema

In [4]:
data = {
    "id": [1, 2],
    "name": ["Alice", "Bob"],
    "email": ["a@example.com", None],
    "is_active": [True, False],
    "address": [
        {"street": "123 Main St", "city": "NYC", "zip_code": 10001},
        {"street": "456 Side St", "city": "LA", "zip_code": None},
    ],
    "tags": [["admin"], ["editor", "reviewer"]],
}

df = pl.DataFrame(data)
print("Polars DataFrame:")
print(df)

print("\nInferred Polars schema:")
print(df.schema)

Polars DataFrame:
shape: (2, 6)
┌─────┬───────┬───────────────┬───────────┬─────────────────────────────┬────────────────────────┐
│ id  ┆ name  ┆ email         ┆ is_active ┆ address                     ┆ tags                   │
│ --- ┆ ---   ┆ ---           ┆ ---       ┆ ---                         ┆ ---                    │
│ i64 ┆ str   ┆ str           ┆ bool      ┆ struct[3]                   ┆ list[str]              │
╞═════╪═══════╪═══════════════╪═══════════╪═════════════════════════════╪════════════════════════╡
│ 1   ┆ Alice ┆ a@example.com ┆ true      ┆ {"123 Main St","NYC",10001} ┆ ["admin"]              │
│ 2   ┆ Bob   ┆ null          ┆ false     ┆ {"456 Side St","LA",null}   ┆ ["editor", "reviewer"] │
└─────┴───────┴───────────────┴───────────┴─────────────────────────────┴────────────────────────┘

Inferred Polars schema:
Schema({'id': Int64, 'name': String, 'email': String, 'is_active': Boolean, 'address': Struct({'street': String, 'city': String, 'zip_code': Int64}), '